<a href="https://colab.research.google.com/github/Timurus/ABalytics/blob/master/bq_schema_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from dateutil import parser
d2 = [{
    'string':'string_values',
    'bool': 'True',
    'array': ['foo', 'bar'],
    'obj_name': {
        "obj_foo": "bar",
        'obj_in_obj_name': {
            'obj_foo':'1',
            'obj_number':'1'
        }
    }
}]
schema = {
    "string": {"type": "string"},
    'bool':{"type": "boolean"},
    "array": {"type": "array", 'element_type': 'boolean'},
    "obj_name": {
        "type": "object",
        "properties": {
                "obj_foo": {"type": "string"},
                "obj_in_obj_name": {
                  "type": "object",
                    "properties": {
                "obj_foo": {"type": "string"},
                "obj_number": {"type": "integer"},
        }
    }
        }
    }
}


# def dot_get(_dict, path, default=None):
#     for key in path:
#         try:
#             _dict = _dict[key]
#         except KeyError:
#             return default

from functools import reduce  # forward compatibility for Python 3
import operator

def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    assert type(items) == list
    return reduce(operator.getitem, items, root)

def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value

def to_bq_format(val, bq_format):
    bq_format = bq_format.upper()
    if val == None:
        val = None
    elif bq_format == 'STRING':
        val = str(val)
    elif bq_format == 'BOOLEAN':
        val = bool(val)
    elif bq_format in ['INTEGER']:
        val = int(val)
    elif bq_format == 'FLOAT':
        val = float(val)
    elif bq_format == 'TIMESTAMP':
        try:
            val = parser.parse(val).strftime("%Y-%m-%d %H:%M:%S")
        except:
            val = None
    return val


result = {}
def walk_obj_by_shema(schema, obj, schema_path = [], path=[]):
    for k, v in get_by_path(schema,schema_path).items():
        z = 0
        this_schema = get_by_path(schema,schema_path+[k])
        schema_el_type = this_schema['type']
        if schema_el_type == 'object':
            new_path = path+[k]
            new_shema_path = schema_path + [k, 'properties']
            set_by_path(result, path+[k], {})

            walk_obj_by_shema(schema, obj, new_shema_path, new_path)
            return
        else:
            original_json_value = get_by_path(obj, path+[k])
            if schema_el_type == 'array':
              # assert type(original_json_value) in (int, float, bool, str), 'Dict in liss not suporte'

              formatted_value = [to_bq_format(e, this_schema['element_type']) for e in original_json_value]
            else:  
              formatted_value = to_bq_format(original_json_value, schema_el_type)
            set_by_path(result, path+[k], formatted_value)
    print(result)        
    return result


walk_obj_by_shema(schema, d2[0] )



{'string': 'string_values', 'bool': True, 'array': [True, True], 'obj_name': {'obj_foo': 'bar', 'obj_in_obj_name': {'obj_foo': '1', 'obj_number': 1}}}
